In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("/home/dux/")
sys.path.append("/home/dux/surface_sampling/mcmc")

from ase.build import make_supercell
from ase.io import read, write

import catkit
from catkit.gen.adsorption import get_adsorption_sites

from mcmc import mcmc_run
import os
import numpy as np

os.environ["PROJECT_DIR"] = os.getcwd()

In [3]:
# SrTiO3 from HTVS db
import sys
import os
import django
import numpy as np
# from tqdm import tqdm
# import pandas as pd
# import pickle as pkl
sys.path.append('/home/dux/htvs/')
sys.path.append('/home/dux/htvs/djangochem/')

#from imports import *
#from django_imports import *
# from pymatgen.core.periodic_table import Element
# from chemconfigs.vasp.defaults import Magmom
# from ase.visualize import view

# setup the django settings file.  Change this to use the settings file that connects you to your desired database
os.environ["DJANGO_SETTINGS_MODULE"] = "djangochem.settings.orgel"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"  # CAUTION!!! LINE NECESSARY WHEN INTERACTING WITH DATABASE IN JUPYTER NOTEBOOK. DO NOT USE THIS LINE IF YOU ARE PLANNING ON SAVING/REMOVING THINGS FROM THE DATABASE!!!!!

# this must be run to setup access to the django settings and make database access work etc.
django.setup()
# from jobs.models import Job, JobConfig
# from django.contrib.contenttypes.models import ContentType
# from ase.io import write

from pgmols.models import Surface
# from analysis.structure.perovskite import PerovskiteAnalyzer
# from pymatgen.core.periodic_table import Element

unit_slab = Surface.objects.get(id=217485271).as_ase_gratoms()
unit_slab.write('SrTiO3_001_pristine_slab.cif')

# get supercell
# multiplier = np.diag([2, 2, 1])
# slab = make_supercell(unit_slab, multiplier)
slab = unit_slab*(2,2,1)
slab.write('SrTiO3_001_2x2_pristine_slab.cif')

In [4]:
# Pymatgen adsorption sites for InGaN
from pymatgen.analysis.adsorption import AdsorbateSiteFinder
from pymatgen.io.ase import AseAtomsAdaptor

from ase.io import read, write

pristine_slab = slab
pristine_pmg_slab = AseAtomsAdaptor.get_structure(pristine_slab)
site_finder = AdsorbateSiteFinder(pristine_pmg_slab)
# AdsorbateSiteFinder(slab, selective_dynamics=False, height=0.9, mi_vec=None)
sites = site_finder.find_adsorption_sites(put_inside=False, symm_reduce=False)
# find_adsorption_sites(distance=2.0, put_inside=True, symm_reduce=0.01, near_reduce=0.01, positions=['ontop', 'bridge', 'hollow'], no_obtuse_hollow=True)

In [5]:
for type in sites.keys():
    print(f"{type} sites has total {len(sites[type])}")

ontop sites has total 12
bridge sites has total 36
hollow sites has total 16
all sites has total 64


In [6]:
ads_positions = sites['all']
# print(f"adsorption coordinates are")
# print(ads_positions)
# take just the first 18 sites after inspection
# select_positions = ads_positions[:18]

# take all positions
select_positions = ads_positions

In [7]:
# adsorb onto the sites
from pymatgen.core.structure import Molecule
ads_mol = Molecule(['O'], [[0, 0, 0]])
for ads_coord in select_positions:
    slab = site_finder.add_adsorbate(ads_mol, ads_coord, repeat=None, translate=False, reorient=False)
    site_finder.slab = slab

from pymatgen.io.cif import CifWriter
# write out slab
cif_slab = CifWriter(site_finder.slab)
# cif_slab.write_file("pymatgen_SrTiO3_001_4x4_O_all_adsorbed_slab.cif")
cif_slab.write_file("pymatgen_SrTiO3_001_2x2_O_all_adsorbed_slab.cif")

In [8]:
all_adsorbed = read("pymatgen_SrTiO3_001_2x2_O_all_adsorbed_slab.cif")
print(f"all adsorbed {len(all_adsorbed)}")

select_positions = all_adsorbed.get_positions()[len(pristine_slab):]

all adsorbed 124


In [16]:
# set logging level to DEBUG for mcmc.py
import logging
logging.getLogger('mcmc.mcmc').setLevel(logging.DEBUG)

# so add a pristine slab
element = 'O'
filter_distance = 1.5 # angstroms
num_sweeps = 20

chem_pot = 0
alpha = 1
slab = pristine_slab.copy()
surface_name = f"pymatgen_SrTiO3_001_2x2"

ads_positions = select_positions

import logging
logging.basicConfig(
    format="%(levelname)s:%(message)s",
    level=logging.DEBUG,
    datefmt="%m/%d/%Y %I:%M:%S %p",
)

history, energy_hist, frac_accept_hist, adsorption_count_hist = mcmc_run(num_sweeps=num_sweeps, temp=1, pot=0, alpha=alpha, slab=slab, calc=None, testing=True, surface_name=surface_name, element=element, ads_coords=ads_positions, filter_distance=filter_distance)

Test to ensure that the slabs indeed make sense

In [48]:
pristine_slab.get_surface_atoms()

array([ 7,  8, 14, 22, 23, 29, 37, 38, 44, 52, 53, 59])

In [49]:
surface_atoms = pristine_slab.get_surface_atoms()
atoms_arr = np.arange(0, len(pristine_slab))
base_tags = np.int8(np.isin(atoms_arr, surface_atoms)).tolist()

In [50]:
# set tags; 1 for surface atoms, 2 for adsorbates, 0 for others
for slab in history:
    num_ads_atoms = len(slab) - len(pristine_slab)
    tags = list(base_tags)
    tags.extend([2]*num_ads_atoms)
    slab.set_tags(tags)

In [51]:
for s in history:
    print(s.get_tags())
    print(f"Oxygen adsorbates are outside distance cutoffs {filter_distances(s, pristine_len=len(pristine_slab), cutoff_distance=1.5)}")

[0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0
 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 2 2 2 2 2 2 2 2 2]
Oxygen adsorbates are outside distance cutoffs True
[0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0
 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 2 2 2 2 2 2 2 2]
Oxygen adsorbates are outside distance cutoffs True
[0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0
 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 2 2 2 2 2 2 2 2 2 2]
Oxygen adsorbates are outside distance cutoffs True
[0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0
 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 2 2 2 2 2 2 2 2 2 2 2 2]
Oxygen adsorbates are outside distance cutoffs True
[0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0
 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 2 2 2 2 2 2 2 2 2 2]
Oxygen adsorbates are outside distance cutoffs True
[0 0 0 0 0 0 0 1 1 0 0 0 0 

Save to .pkl

In [52]:
import pickle
print(f"slab length {len(history)}")
with open("pymatgen_SrTiO3_001_2x2_O_ads_2022_0706_20_samples.pkl", "wb") as f:
    pickle.dump(history, f)

slab length 20
